In [ ]:
import sqlite3
import csv

This file is currently just for testing purposes---plays with a placeholder sqlite3 database that has columns "test_1" and "test_2"

In [ ]:
with open("./stored_authors/root_names.txt", 'w') as root_out:
    root_names = eval(root_out.read())
with open("./stored_authors/mergees.txt", 'w') as mergees_out:
    mergees = eval(mergees_out.read())

In [ ]:
def select_and_print(cursor, command_string):
    """
    Selects entries from database and prints it out to the python interpreter. 
    
    Parameters: 
    cursor         : Sqlite cursor object - the sqlite3 cursor created from .cursor() on a connected sqlite 3 database
    command_string : String               - the entire select SQL command (e.g. "select * from testing123;")
    """
    
    cursor.execute(command_string)

    rows = cursor.fetchall()
    for row in rows:
        print(row.keys())

        string = ""
        for key in row.keys():
            string += str(row[key])
            string += " | "
        # print(str(row['test_1']) + " | " + str(row['test_2']))

        print(string)
        
    # print(rows.keys())
    
    # print(rows[0]['test_1'])
    # print(rows[0]['test_2'])
    print("---------------")

In [ ]:
# Incomplete, currently only concatenates data string together and does nothing to modify the database.
def update_values_test(cursor, root_id, mergee_ids):
    """
    Merges entries into one master or root entry by combining their values (strings) into a merged_string.
    
    Parameters:
    cursor      : Sqlite cursor object - the sqlite3 cursor created from .cursor() on a connected sqlite 3 database
    root_id     : String               - the ID of the "root" entry, or the entry that absorbs all of the other entries being merged
    mergee_ids  : String               - list of IDs of the entries that will be merged into the root entry
    """
    
    # Extracts string from root entry to use as the base of the merged_string
    select_root = "select test_2 from testing123 where test_1 = " + str(root_id)
    cursor.execute(select_root)
    merged_string = cursor.fetchone()[0]
    
    # Iterates through mergees to merge them into the root
    for mergee_id in mergee_ids:
        # Extracts string from mergee and adds it to the merged_string
        select_mergee = "select test_2 from testing123 where test_1 = " + str(mergee_id)
        cursor.execute(select_mergee)
        merged_string += cursor.fetchone()[0]
        
        # Deletes the mergee 
        delete_command = "delete from testing123 where test_1 = " + mergee_id
        cursor.execute(delete_command)
    
    # Updates entry of root to be merged string
    update_command = "update testing123 set test_2 = '" + merged_string + "' where test_1 = " + root_id
    cursor.execute(update_command)
    
    print(merged_string)

In [ ]:
def update_values(cursor, root_id, mergee_ids, new_root_name):
    # Updates given, middle, and family names of root_id
    string_tokens = new_root_name.split('_')
    
    root_name_update = "update DBADSORPTION_SANDBOX.authors set given_name = '" + string_tokens[0] + "' where author_id = '" + root_id + "';"
    cursor.execute(root_name_update)
    root_name_update = "update DBADSORPTION_SANDBOX.authors set middle_name = '" + string_tokens[1] + "' where author_id = '" + root_id + "';"
    cursor.execute(root_name_update)
    root_name_update = "update DBADSORPTION_SANDBOX.authors set family_name = '" + string_tokens[2] + "' where author_id = '" + root_id + "';"
    cursor.execute(root_name_update)
    
    # Iterates through mergees and changes each foreign key in biblio_authors to the root_id 
    for mergee in mergee_ids:
        foreign_change_string = "update DBADSORPTION_SANDBOX.biblio_authors set author_id = '" + root_id + "' where author_id = '" +  mergee + "';"
        cursor.execute(foreign_change_string)
        delete_string = "DELETE from DBADSORPTION_SANDBOX.authors WHERE author_id = '" + mergee + "';"
        cursor.execute(delete_string)

In [ ]:
for root, pair in mergees.items():
    update_values(cursor, root, pair, root_names[root])